In [ ]:
import cv2
import numpy as np
import math
import time as time
import random as rand

In [ ]:
class Bird:
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y
        self.gravity = 0.981
        self.jump_amount = 10 * self.gravity
        self.acc = 0
        self.vel = 0
        self.alive = True
        self.horiz_dist_travel = 0

    def draw(self, canvas):
        cv2.putText(canvas, '({}, {})'.format(self.x, self.y), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))
        cv2.circle(canvas, (self.x, self.y), radius=3, color=(255, 0, 0), thickness=-1)
        # draw circle on canvas to represent bird location
        
    def update_motion(self):
        self.acc = self.gravity
        self.vel += self.acc
        self.y = int(self.y + self.vel)
        self.horiz_dist_travel += 1
        
    def jump(self):
        self.vel = -self.jump_amount
        
    def get_next_move(self):
        self.jump()
            

In [ ]:
class GeneticPlayer():
    def __init__(self, pop_size=100, num_generations=100):
        self.pop_size = pop_size
        self.num_generations = num_generations
        
        # inputs => bird y, obstacle x, obstacle height_top, obstacle height_bot,
        input_size = 4
        hidden_size1 = 10
        hidden_size2 = 10
        output_size = 1
        self.layer_sizes = [input_size, hidden_size1, hidden_size2, output_size]
        
        # mutation
        self.mut_rate = 0.1
        self.mut_size = 0.1
        
        self.pop = [Brain(self.layer_sizes) for _ in range(self.pop_size)]
        
        self.init_x = 50
        self.init_y = 250
        
        self.last_best_brain = None
            
    def run(self):
        max_score = 0
        scores = []
        # loop through each brain
        for i, brain in enumerate(self.pop):
            # run game using current self (bird with brain)
            bird = GeneticBird(self.init_x, self.init_y, brain)
            game = Game(bird)
            game.run()
            # get score (= count)
            score = bird.horiz_dist_travel
            if score > max_score:
                self.last_best_brain = brain
                max_score = score
                print('Highest score so far: {} for brain: {}'.format(score, i))
            # store score
            scores.append(score)
        # isolate for top scores anad select
        candid_indices = list(np.argsort(scores))[-self.pop_size // 4:] # get top 25%
        candids = [self.pop[i] for i in candid_indices][::-1] # reverse list so top brain is first
        # mutate, pick randomly
        # set to current population
        self.pop = self.selection(candids)
        
    def selection(self, candids):
        new_pop = []
        
        for i in range(len(candids)):
            new_pop.append(candids[i])
            new_pop.append(candids[i].mutate(self.mut_rate, self.mut_size))
            
        for i in range(self.pop_size - len(new_pop)):
            new_pop.append(Brain(self.layer_sizes))
            
        return new_pop
        
    def evolve(self):
        # loop through each generation
        for i in range(self.num_generations):
            print('Generation: {}'.format(i))
            self.run()
            # run
    

In [ ]:
class GeneticBird(Bird):
    def __init__(self, x, y, brain):
        Bird.__init__(self, x, y)
        self.current_brain = brain
            
    def get_next_move(self, game):
        # inputs => bird y, obstacle x, obstacle height_top, obstacle height_bot,
        obs = game.obstacles[game.cur_obs_ind]
        inputs = [self.y, obs.x, obs.height_top, obs.height_bot]
        out = self.current_brain.feedforward(inputs)
        
        if out >= 0.5:
            # jump
#             print('Jump')
            self.jump()
        else:
            pass
        

In [ ]:
class Brain:
    def __init__(self, layer_sizes):
        self.input_size = layer_sizes[0]
        self.layer_sizes = layer_sizes[1:] # remove input size layer
        self.layers = []
        for i in range(0, len(layer_sizes) - 1):
            new_layer = np.array([[rand.uniform(-1, 1) for _ in range(layer_sizes[i] + 1)] for _ in range(layer_sizes[i + 1])])
            self.layers.append(new_layer)
        
    def mutate(self, mut_rate, mut_size):
        layer_sizes = [self.input_size] + self.layer_sizes
        new_brain = Brain(layer_sizes)
        for l in range(len(new_brain.layer_sizes)):
            for i in range(len(new_brain.layers[l])):
                for j in range(len(new_brain.layers[l][i])):
                    if rand.uniform(0, 1) < mut_rate:
                        new_brain.layers[l][i][j] += rand.uniform(-1, 1) * mut_size
        return new_brain
    
    def feedforward(self, inputs):
        cur_result = inputs + [1]
        for i in range(len(self.layers) - 1):
            next_result = np.array([math.tanh(np.dot(cur_result, self.layers[i][j])) for j in range(self.layers[i].shape[0])] + [1])
            cur_result = next_result
    
        output_result = np.array([math.tanh(np.dot(cur_result, self.layers[-1][j])) for j in range(self.layers[-1].shape[0])])
        
        # apply sigmoid
        raw_output = output_result[0]
        output = 1 / (1 + math.e**(-raw_output))
        return output

In [ ]:
class Obstacle:
    def __init__(self, x, width, height_top, height_bot):
        self.x = x
        self.width = width
        self.height_top = height_top
        self.height_bot = height_bot
    
    def draw(self, canvas):
        height, width = canvas.shape
        cv2.rectangle(canvas, (self.x, 0), (self.x + self.width, self.height_top), (255, 0, 0), -1)
        cv2.rectangle(canvas, (self.x, height - self.height_bot), (self.x + self.width, height), (255, 0, 0), -1)
        # draw obstacle on canvas 

    def update_motion(self):
        self.x -= 2

In [ ]:
def collision(bird, obs, canvas_height):
#     print('obstacle: x: {} | width: {} | y_top: {} | y_bot: {}'.format(obs.x, obs.width, obs.height_top, canvas_height - obs.height_bot))
    return bird.x >= obs.x and bird.x <= obs.x + obs.width and (bird.y <= obs.height_top or bird.y >= canvas_height - obs.height_bot)

In [ ]:
class Game:
    def __init__(self, bird=None):
        # set up canvas
        self.canvas_width = 500
        self.canvas_height = 500
        self.canvas = np.zeros((self.canvas_height, self.canvas_width))
        
        # set up bird
        self.bird_init_x = 50
        self.bird_init_y = self.canvas_height // 2
        if bird is None:
            self.bird = Bird(self.bird_init_x, self.bird_init_y)
        else:
            self.bird = bird
            self.bird.x = self.bird_init_x
            self.bird.y = self.bird_init_y
        
        # setup obstacle
        self.min_obs_height = 10
        self.obs_offset = 100
        self.obs_gap = 100
        self.obs_width = 20
        self.heights = [(113, 287), (386, 14), (188, 212), (362, 38), (151, 249), (322, 78), (360, 40), (120, 280), (227, 173), (362, 38), (345, 55), (240, 160), (259, 141), (118, 282), (140, 260), (202, 198), (319, 81), (189, 211), (333, 67), (367, 33), (97, 303), (389, 11), (287, 113), (48, 352), (204, 196), (280, 120), (46, 354), (100, 300), (58, 342), (350, 50), (104, 296), (264, 136), (349, 51), (129, 271), (11, 389), (158, 242), (35, 365), (124, 276), (279, 121), (257, 143), (154, 246), (263, 137), (364, 36), (160, 240), (170, 230), (246, 154), (187, 213), (351, 49), (82, 318), (171, 229), (263, 137), (355, 45), (29, 371), (293, 107), (288, 112), (129, 271), (270, 130), (382, 18), (244, 156), (93, 307), (260, 140), (29, 371), (183, 217), (162, 238), (99, 301), (182, 218), (355, 45), (311, 89), (130, 270), (226, 174), (241, 159), (273, 127), (267, 133), (249, 151), (384, 16), (185, 215), (52, 348), (262, 138), (72, 328), (344, 56), (45, 355), (246, 154), (133, 267), (296, 104), (80, 320), (232, 168), (331, 69), (253, 147), (269, 131), (289, 111), (305, 95), (280, 120), (378, 22), (358, 42), (121, 279), (337, 63), (134, 266), (14, 386), (24, 376), (274, 126), (113, 287), (364, 36), (72, 328), (351, 49), (168, 232), (326, 74), (34, 366), (30, 370), (122, 278), (19, 381), (88, 312), (220, 180), (188, 212), (305, 95), (225, 175), (298, 102), (61, 339), (89, 311), (263, 137), (319, 81), (203, 197), (362, 38), (158, 242), (207, 193), (350, 50), (111, 289), (158, 242), (250, 150), (315, 85), (244, 156), (198, 202), (105, 295), (26, 374), (305, 95), (120, 280), (255, 145), (323, 77), (43, 357), (236, 164), (213, 187), (126, 274), (23, 377), (372, 28), (226, 174), (173, 227), (390, 10), (178, 222), (25, 375), (220, 180), (134, 266), (223, 177), (73, 327), (228, 172), (170, 230), (363, 37), (227, 173), (337, 63), (55, 345), (142, 258), (226, 174), (349, 51), (313, 87), (65, 335), (222, 178), (126, 274), (52, 348), (95, 305), (335, 65), (366, 34), (334, 66), (142, 258), (143, 257), (385, 15), (148, 252), (147, 253), (323, 77), (160, 240), (338, 62), (29, 371), (134, 266), (82, 318), (314, 86), (195, 205), (139, 261), (251, 149), (366, 34), (143, 257), (321, 79), (162, 238), (352, 48), (67, 333), (151, 249), (331, 69), (158, 242), (237, 163), (295, 105), (329, 71), (314, 86), (268, 132), (58, 342)]
        self.stored_obstacles = [Obstacle(self.canvas_width - self.obs_width, self.obs_width, height_top, height_bot) for (height_top, height_bot) in self.heights]
        self.stored_obstacle_index = 1
        self.obstacles = [self.get_next_obstacle()]
        self.cur_obs_ind = 0
        
#         heights2 = []
#         for i in range(200):
#             height_top = rand.randint(self.min_obs_height, self.canvas_height - self.min_obs_height - self.obs_gap)
#             height_bot = self.canvas_height - height_top - self.obs_gap
#             heights2.append((height_top, height_bot))
#         print(heights2)
        
        self.count = 0
    
    def get_next_obstacle(self):
        return self.stored_obstacles.pop(0)
        
    def draw(self):
        self.bird.draw(self.canvas)
        for obs in self.obstacles:
            obs.draw(self.canvas)
        
        # display actual canvas
        cv2.putText(self.canvas, 'Count: {}'.format(self.count), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))
        cv2.imshow('canvas', self.canvas)
        
    def update(self):
        self.bird.update_motion()
        for obs in self.obstacles:
            obs.update_motion()
            
        if collision(self.bird, self.obstacles[self.cur_obs_ind], self.canvas_height) or self.cur_obs_ind != 0 and collision(self.bird, self.obstacles[0], self.canvas_height):
#             print('Game Over')
            self.bird.alive = False
        else:
            pass
#             print('Bird: alive | x: {} | y: {}'.format(self.bird.x, self.bird.y))
         
        if self.obstacles[-1].x <= self.canvas_width - (2 * self.obs_width  + self.obs_offset):
            self.obstacles.append(self.stored_obstacles[self.stored_obstacle_index])
            self.stored_obstacle_index += 1
        
        if self.obstacles[0].x <= -self.obs_width:
            self.obstacles.pop(0)
            self.cur_obs_ind -= 1
        
        if self.obstacles[self.cur_obs_ind].x <= self.bird.x:
            self.count += 1
            self.cur_obs_ind += 1
        
    def run(self, draw=False):
        while True:
            if draw:
                self.draw()
    #             clear canvas
                del self.canvas
                self.canvas = np.zeros((self.canvas_height, self.canvas_width))
            
            self.update()
            
            if not self.bird.alive:
                break
            
            self.bird.get_next_move(self)
            
            if draw:
                k = cv2.waitKey(30) & 0xFF
                if k == ord('q'):
                    break
    #             elif k == ord(' '): # jump
    #                 self.bird.jump()
                
        cv2.destroyAllWindows()

In [ ]:
player = GeneticPlayer(pop_size=100, num_generations=100)

In [ ]:
# game = Game()
# game.run()
player.num_generations = 1000
player.evolve()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
game = Game(GeneticBird(50, 250, player.pop[0]))
game.run(draw=True)

In [ ]:
math.e